<a href="https://colab.research.google.com/github/zloelias/final-project/blob/master/finetune_kinopoisk_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install transformers
! pip install datasets

     |████████████████████████████████| 3.1 MB 9.0 MB/s 
     |████████████████████████████████| 596 kB 62.3 MB/s 
     |████████████████████████████████| 895 kB 67.8 MB/s 
     |████████████████████████████████| 3.3 MB 69.2 MB/s 
     |████████████████████████████████| 61 kB 639 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 298 kB 8.2 MB/s 
     |████████████████████████████████| 243 kB 51.9 MB/s 
     |████████████████████████████████| 1.1 MB 74.0 MB/s 
     |████████████████████████████████| 132 kB 84.9 MB/s 
     |████████████████████████████████| 160 kB 75.9 MB/s 
     |████████████████████████████████| 192 kB 87.7 MB/s 
     |████████████████████████████████| 271 kB 66.2 MB/s 


In [3]:
! git config --global credential.helper store

In [4]:
! huggingface-cli login


        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/token.
        (Deprecated, will be removed in v0.3.0) To login with username and password instead, interrupt with Ctrl+C.
        
Token: 
Login successful
Your token has been saved to /root/.huggingface/token


In [5]:
! curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
! sudo apt-get install git-lfs
! git lfs install

Detected operating system as Ubuntu/bionic.
Checking for curl...
Detected curl...
Checking for gpg...
Detected gpg...
Running apt-get update... done.
Installing apt-transport-https... done.
Installing /etc/apt/sources.list.d/github_git-lfs.list...done.
Importing packagecloud gpg key... done.
Running apt-get update... done.

The repository is setup! You can now install packages.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 43 not upgraded.
Need to get 6,526 kB of archives.
After this operation, 14.7 MB of additional disk space will be used.
Get:1 https://packagecloud.io/github/git-lfs/ubuntu bionic/main amd64 git-lfs amd64 3.0.2 [6,526 kB]
Fetched 6,526 kB in 0s (19.6 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl

In [6]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer



In [7]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else: 
    device = torch.device("cpu")
    
print(device)

cuda:0


In [12]:
from datasets import load_dataset

dataset = load_dataset('zloelias/kinopoisk-reviews')

Using custom data configuration zloelias___kinopoisk-reviews-62e5c303d108c552


  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/258M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.7M [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/parquet/zloelias___kinopoisk-reviews-62e5c303d108c552/0.0.0/1638526fd0e8d960534e2155dc54fdff8dce73851f21f031d2fb9c2cf757c121. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
def count_classes(dataset, label_key='labels'):
    classes = set()
    dataset['train'].map(lambda x: classes.add(x[label_key]))
    return len(classes)

In [10]:
from transformers import BertTokenizer
from transformers import BertForSequenceClassification

BERT_MODEL = "cointegrated/rubert-tiny2"

tokenizer = BertTokenizer.from_pretrained(BERT_MODEL, padding=True, truncation=True)
model = BertForSequenceClassification.from_pretrained(BERT_MODEL, num_labels = count_classes(dataset))
model.to(device)

Downloading:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/401 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.66M [00:00<?, ?B/s]

  0%|          | 0/118502 [00:00<?, ?ex/s]

Downloading:   0%|          | 0.00/715 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112M [00:00<?, ?B/s]

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not 

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(83828, 312, padding_idx=0)
      (position_embeddings): Embedding(2048, 312)
      (token_type_embeddings): Embedding(2, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=312, out_features=312, bias=True)
              (key): Linear(in_features=312, out_features=312, bias=True)
              (value): Linear(in_features=312, out_features=312, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=312, out_features=312, bias=True)
              (LayerNorm): LayerNorm((312,), eps=1e-12, elemen

In [13]:
from transformers import TrainingArguments, Trainer

batch_size = 8

def preprocess_function(examples, sentence_key='text'):
    return tokenizer(examples[sentence_key], padding=True, truncation=True, max_length=512)

dataset = dataset.map(preprocess_function, batched=True)

#dataset['train'] = dataset['train'].select(list(range(0,1800))).map(preprocess_function, batched=True)
#dataset['test'] = dataset['test'].select(list(range(0,300))).map(preprocess_function, batched=True)

  0%|          | 0/119 [00:00<?, ?ba/s]

  0%|          | 0/14 [00:00<?, ?ba/s]

In [14]:
metric_name = "accuracy"
dataset_name = "kinopoisk-reviews"
task = "clf"
model_name = BERT_MODEL.split("/")[-1]

batch_size = 32

args = TrainingArguments(
    f"{model_name}-{dataset_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=10,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=True,
)

from sklearn.metrics import accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
    }

trainer = Trainer(
    model,
    args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Cloning https://huggingface.co/zloelias/rubert-tiny2-kinopoisk-reviews-finetuned-clf into local empty directory.


In [15]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: label_name, text, __index_level_0__.
***** Running training *****
  Num examples = 118502
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 320
  Gradient Accumulation steps = 10
  Total optimization steps = 1850


Epoch,Training Loss,Validation Loss,Accuracy
0,No log,0.543350,0.776866
1,0.661400,0.513176,0.785676
2,0.510900,0.504420,0.791145
3,0.510900,0.502197,0.791524
4,0.489300,0.501433,0.792056


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: label_name, text, __index_level_0__.
***** Running Evaluation *****
  Num examples = 13167
  Batch size = 32
Saving model checkpoint to rubert-tiny2-kinopoisk-reviews-finetuned-clf/checkpoint-370
Configuration saved in rubert-tiny2-kinopoisk-reviews-finetuned-clf/checkpoint-370/config.json
Model weights saved in rubert-tiny2-kinopoisk-reviews-finetuned-clf/checkpoint-370/pytorch_model.bin
tokenizer config file saved in rubert-tiny2-kinopoisk-reviews-finetuned-clf/checkpoint-370/tokenizer_config.json
Special tokens file saved in rubert-tiny2-kinopoisk-reviews-finetuned-clf/checkpoint-370/special_tokens_map.json
tokenizer config file saved in rubert-tiny2-kinopoisk-reviews-finetuned-clf/tokenizer_config.json
Special tokens file saved in rubert-tiny2-kinopoisk-reviews-finetuned-clf/special_tokens_map.json
The following columns in the evaluation

TrainOutput(global_step=1850, training_loss=0.5395924583641258, metrics={'train_runtime': 3410.7331, 'train_samples_per_second': 173.719, 'train_steps_per_second': 0.542, 'total_flos': 4369108827856896.0, 'train_loss': 0.5395924583641258, 'epoch': 5.0})